In [ ]:
# %%
# %% [0] (optional) allow big uploads + quiet analytics
import os
os.environ.setdefault("GRADIO_MAX_UPLOAD_SIZE", "512")  # MB
os.environ.setdefault("GRADIO_ANALYTICS_ENABLED", "0")

In [ ]:
# %%
# %% [1] Imports, env, model
import os, json, time, html, re
from dotenv import load_dotenv
from PIL import Image, ImageOps, ImageDraw
import gradio as gr
from openai import OpenAI
from pathlib import Path  

# Directory where  sample images live
SAMPLE_DIR = Path("sample_images")   

SAMPLE_FILES = [
    SAMPLE_DIR / "bottles.jpg",
    SAMPLE_DIR / "burger.jpg",
    SAMPLE_DIR / "car.jpg",
    SAMPLE_DIR / "man.jpg",
    SAMPLE_DIR / "vegetables.jpg",
]

# Load once as PIL images
SAMPLE_IMAGES = [Image.open(p).convert("RGB") for p in SAMPLE_FILES]

load_dotenv("api.env")
API_KEY  = os.getenv("MOONDREAM_API_KEY", "").strip()
ENDPOINT = os.getenv("MOONDREAM_ENDPOINT", "http://localhost:2020/v1").strip()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "").strip()
OA_CLIENT = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None

import moondream as md
MODEL  = md.vl(api_key=API_KEY) if API_KEY else md.vl(endpoint=ENDPOINT)
SOURCE = "moondream-cloud" if API_KEY else "moondream-local"


In [ ]:
# %%
# %% [2] Helpers

def exif_rgb(img: Image.Image, max_side: int = 1024) -> Image.Image:
    """
    Convert to RGB, fix EXIF orientation, and downscale so the longest side
    is at most `max_side` pixels. This dramatically cuts payload + compute
    without hurting captions much.
    """
    img = ImageOps.exif_transpose(img.convert("RGB"))
    w, h = img.size
    longest = max(w, h)
    if longest > max_side:
        scale = max_side / float(longest)
        new_size = (int(w * scale), int(h * scale))
        img = img.resize(new_size, Image.LANCZOS)
    return img


def to_px_box(obj, w, h):
    return [
        int(obj["x_min"] * w),
        int(obj["y_min"] * h),
        int(obj["x_max"] * w),
        int(obj["y_max"] * h),
    ]


def to_px_point(p, w, h):
    return [int(p["x"] * w), int(p["y"] * h)]


def jdump(obj) -> str:
    return json.dumps(obj, indent=2, ensure_ascii=False)


def make_code_details(json_text: str) -> str:
    # Disable the "Show Code" block entirely
    return ""



def _steer(q: str) -> str:
    return (
        "Answer the question in one clear, complete sentence. "
        "If you’re unsure, say you’re unsure and why. "
        "Avoid deictic words like 'this' or 'that'.\n\n"
        f"Question: {q.strip()}"
    )


def _is_low_info(ans: str) -> bool:
    a = (ans or "").strip().lower()
    return (len(a) <= 4) or a in {"this", "that", "yes", "no", "unknown", "n/a"}


# Styled overlays like the playground
def draw_overlay(base: Image.Image, detections=None, points=None) -> Image.Image:
    detections = detections or []
    points = points or []
    canvas = base.convert("RGBA")
    overlay = Image.new("RGBA", canvas.size, (0, 0, 0, 0))
    d = ImageDraw.Draw(overlay)

    RED = (239, 68, 68, 255)   # red-500
    for det in detections:
        x1, y1, x2, y2 = [int(v) for v in det["box"]]
        d.rectangle([x1, y1, x2, y2], outline=RED, width=3)

    BLUE = (59, 130, 246)      # blue-500
    for pt in points:
        x, y = [int(v) for v in pt["xy"]]
        # glow
        r_glow = 16
        d.ellipse([x-r_glow, y-r_glow, x+r_glow, y+r_glow],
                  fill=(BLUE[0], BLUE[1], BLUE[2], 70))
        # blue ring
        r_ring = 11
        d.ellipse([x-r_ring, y-r_ring, x+r_ring, y+r_ring],
                  outline=(BLUE[0], BLUE[1], BLUE[2], 255), width=3)
        # thin white ring
        r_inner = 7
        d.ellipse([x-r_inner, y-r_inner, x+r_inner, y+r_inner],
                  outline=(255, 255, 255, 220), width=2)
        # center dot
        r_dot = 3
        d.ellipse([x-r_dot, y-r_dot, x+r_dot, y+r_dot],
                  fill=(255, 255, 255, 255))

    return Image.alpha_composite(canvas, overlay).convert("RGB")


# Output panel sections (PROMPT / REASONING / RESULT)
def _panel_sections(mode: str, sections: dict[str, str], meta: dict,
                    extra_chip: str | None = None) -> str:
    chips = [f"<span class='chip'>{mode.upper()}</span>"]
    if extra_chip:
        chips.append(f"<span class='chip chip--muted'>{html.escape(extra_chip)}</span>")

    rt_ms = meta.get("ms")
    rt_html = (
        f"<div class='meta'>Response time: {int(rt_ms)}ms</div>"
        if isinstance(rt_ms, (int, float)) else ""
    )

    parts = [f"<div class='chips'>{''.join(chips)}</div>"]
    for title, body in sections.items():
        if not body:
            continue
        safe = html.escape(body)
        parts.append(f"<div class='section-h'>{title}</div>")
        parts.append(f"<div class='body'>{safe}</div>")
    parts.append(rt_html)
    return f"<div class='panel'>{''.join(parts)}</div>"


# Caption helper with length control
def caption_with_length(image: Image.Image, length: str) -> tuple[str, dict]:
    t0 = time.time()
    try:
        text = MODEL.caption(image, length=length)["caption"]
    except Exception:
        base = MODEL.caption(image)["caption"]
        if length == "short":
            prompt = (
                "Rewrite this caption into EXACTLY 1 sentence. "
                "No headings or line breaks. Caption: {base}"
            )
        elif length == "normal":
            prompt = (
                "Rewrite this caption into EXACTLY 3 sentences as a single paragraph. "
                "No headings or bullet points. Caption: {base}"
            )
        else:
            prompt = (
                "Expand this caption into TWO short paragraphs separated by a blank line. "
                "Use 4 to 6 sentences total across both paragraphs. "
                "Stay factual and grounded in the image; no lists or headings. "
                f"Caption: {base}"
            )
        try:
            text = MODEL.query(image, prompt)["answer"]
        except Exception:
            text = base

    if length == "long" and "\n" not in text.strip():
        sents = re.split(r'(?<=[.!?])\s+', text.strip())
        if len(sents) >= 4:
            mid = max(2, len(sents) // 2)
            text = " ".join(sents[:mid]) + "\n\n" + " ".join(sents[mid:])

    meta = {
        "ms": int((time.time() - t0) * 1000),
        "source": SOURCE,
        "length": length,
    }
    return text, meta


def suggest_questions(image: Image.Image, n: int = 3) -> list[str]:
    """
    Ask Moondream for n short, concrete example questions about the image.
    Returns a list of up to n strings, but never an empty list – it will
    fall back to generic questions if needed.
    """
    if image is None:
        return []

    default_qs = [
        "What are the main objects in this image?",
        "What is the setting or environment?",
        "Is there anything unusual or interesting here?",
    ]

    prompt = (
        "You are helping a user explore this image in an interactive playground. "
        "Analyze the image carefully and propose 3 short, concrete factual questions "
        "the user might ask specifically about THIS image. "
        "Vary the questions so they touch on things like: environment/natural features, "
        "structures/objects, and any visible activity or context if present.\n\n"
        "Respond ONLY with a JSON array of strings, like:\n"
        "[\"Question about the environment\", \"Question about objects\", \"Question about activity\"]"
    )

    try:
        raw = MODEL.query(image, prompt)["answer"].strip()
    except Exception:
        # Model call failed → just use defaults
        return default_qs[:n]

    qs: list[str] = []

    # Try JSON first
    try:
        data = json.loads(raw)
        if isinstance(data, list):
            qs = [str(x).strip() for x in data if str(x).strip()]
    except Exception:
        pass

    # Fallback: split by lines / bullets if JSON parse didn't work
    if not qs:
        parts = re.split(r"[\n;•\-]+", raw)
        qs = [p.strip().strip("0123456789.:- ") for p in parts if len(p.strip()) > 5]

    # Final safety: if we *still* have nothing, use the defaults
    if not qs:
        qs = default_qs

    return qs[:n]

def run_vision_agent(image: Image.Image, question: str) -> str:
    """
    A fully generic vision-reasoning agent that works for ANY image type:
    food, receipts, landscapes, people, objects, documents, etc.
    
    Pipeline:
      0) Caption with Moondream
      1) Planner (GPT-4.1-mini) – decides WHAT to detect, if anything
      2) Optional Moondream detect calls
      3) Final reasoning (GPT-4.1-mini) – uses caption, detections, and world knowledge
    """

    if OA_CLIENT is None:
        return (
            "⚠️ OpenAI API key not configured.\n\n"
            "Set OPENAI_API_KEY in your environment to enable the vision reasoning agent."
        )

    if image is None:
        return "Please upload an image first."
    if not (question or "").strip():
        return "Please enter a question about the image."

    user_q = question.strip()
    base = exif_rgb(image, max_side=1024)

    # --------------------------
    # STEP 0 — BASE CAPTION
    # --------------------------
    try:
        caption = MODEL.caption(base)["caption"].strip()
    except Exception as e:
        caption = f"(caption error: {e})"

    # --------------------------
    # STEP 1 — UNIVERSAL PLANNER
    # --------------------------
    planner_prompt_system = """
You are a planning module for a general-purpose vision agent.

You NEVER see the image itself. You ONLY see:
- A natural-language CAPTION of the image.
- A USER QUESTION about the image.

Tools available to downstream steps:
- detect(image, label): detects instances of a given object noun (e.g., "boat", "person", "tree").
  This is OPTIONAL. Detection is slow and should not be used unless it clearly adds value.

Your job:
  • Understand what the user question is asking.
  • Decide whether object detection would add useful signal.
  • If so, choose up to 6 SHORT NOUN labels (no adjectives) relevant to the question.
  • If detection is unnecessary, return an empty list.

Examples of good labels:
  ["person", "car", "tree"]
  ["burger", "fries", "plate"]
  ["building", "road"]

BAD labels (not allowed):
  ["large person", "red car", "blurry plate"]  # adjectives → forbidden

Your output MUST be ONLY valid JSON:
{
  "intent": "short summary of what the question is trying to understand",
  "detect_labels": ["noun1", "noun2", ...],
  "notes": "1-2 sentences explaining why you picked those labels"
}
"""

    planner_prompt_user = f"""
CAPTION: {caption}

USER QUESTION: {user_q}

Decide what (if anything) should be detected. Output JSON only.
"""

    try:
        plan_resp = OA_CLIENT.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": planner_prompt_system},
                {"role": "user", "content": planner_prompt_user},
            ],
            temperature=0.2,
        )
        plan_raw = (plan_resp.choices[0].message.content or "").strip()
        try:
            plan = json.loads(plan_raw)
        except Exception:
            plan = {}
    except Exception as e:
        plan = {}
        plan_raw = f"(planner failure: {e})"

    detect_labels = plan.get("detect_labels", [])
    if not isinstance(detect_labels, list):
        detect_labels = []
    detect_labels = [str(x).strip() for x in detect_labels if str(x).strip()]

    intent = plan.get("intent", "")
    notes = plan.get("notes", "")

    # --------------------------
    # STEP 2 — RUN DETECTION (OPTIONAL)
    # --------------------------
    detections = []
    if detect_labels:
        for lb in detect_labels:
            try:
                res = MODEL.detect(base, lb)
                objs = res.get("objects", [])
                detections.append({
                    "label": lb,
                    "count": len(objs),
                })
            except Exception as e:
                detections.append({
                    "label": lb,
                    "error": str(e),
                })

    # --------------------------
    # STEP 3 — FINAL REASONING
    # --------------------------

    reasoning_system = """
You are a universal vision reasoning assistant.

You DO NOT see the image. You ONLY know:
- A natural-language caption of the image.
- Optional detection outputs (object counts).
- General world knowledge (e.g., calories of foods, typical sizes, safety, physics, etc.)
- The user question.

Your responsibilities:
  • Provide careful, step-by-step visual reasoning.
  • Use the caption and detections honestly.
  • NEVER hallucinate objects not supported by caption/detections.
  • If the question asks about food, calories, safety, hazards, objects, text, terrain, etc.,
    use general knowledge to approximate answers.
  • If the image content does NOT actually support the question
    (example: user asks about food but the image is a receipt),
    clearly state that and explain why the image cannot answer the question.

Format your response EXACTLY like this:

Reasoning:
- bullet 1
- bullet 2
- bullet 3
(3–6 bullets total)

Answer:
1–3 sentences directly answering the question using the available evidence.
If giving any numerical estimate (e.g., calories), clearly mark it as approximate.
"""

    vision_context = {
        "caption": caption,
        "intent": intent,
        "planner_notes": notes,
        "detect_labels": detect_labels,
        "detections": detections,
    }

    reasoning_user = (
        "vision_context = " + json.dumps(vision_context, ensure_ascii=False)
        + "\n\nUSER QUESTION:\n" + user_q
    )

    try:
        final_resp = OA_CLIENT.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[
                {"role": "system", "content": reasoning_system},
                {"role": "user", "content": reasoning_user},
            ],
            temperature=0.3,
        )
        answer = (final_resp.choices[0].message.content or "").strip()
    except Exception as e:
        answer = f"(reasoning error: {e})"

    # --------------------------
    # RETURN USER-FACING PANEL
    # --------------------------
    header = (
        "### Tools used\n"
        f"- Planner intent: `{intent or '(none)'}`\n"
        f"- Detect labels: {', '.join(detect_labels) if detect_labels else '(none)'}\n"
        f"- Notes: {notes or '(none)'}\n\n"
    )

    return header + answer


In [ ]:
# %%
# %% [3] Core run

def run(mode, img, question, reasoning, cap_len, labels,
        progress=gr.Progress(track_tqdm=True)):
    """
    Returns: (out_img_update, out_panel_update, code_html_update)
    """
    def _updates(img_value=None, panel_html="", code_html="", show=False):
        return (
            gr.update(value=img_value, visible=show),
            gr.update(value=panel_html, visible=show),
            gr.update(value="", visible=show),
        )

    if img is None:
        return _updates(show=False)

    try:
        progress(0.05, desc="Preparing image")
        base = exif_rgb(img, max_side=1024)
        W, H = base.size

        # --- QUERY ---
        if mode == "Query":
            q = (question or "").strip()
            if not q:
                data = {
                    "mode": "query",
                    "text": "",
                    "suggestions": [
                        "What objects are most prominent?",
                        "What is the likely setting or environment?",
                        "Are any brand names or labels visible?",
                    ],
                    "meta": {"source": SOURCE, "ms": 0},
                }
                right = _panel_sections(
                    "query",
                    {"RESULT": "Suggestions: " + " • ".join(data["suggestions"])},
                    data["meta"],
                )
                return _updates(
                    img_value=base,
                    panel_html=right,
                    code_html=make_code_details(jdump(data)),
                    show=True,
                )

            sections = {"PROMPT": q}
            reason_ms = 0

            if reasoning:
                progress(0.25, desc="Thinking through the question")
                t_r = time.time()
                rationale = MODEL.query(
                    base,
                    "Briefly explain the reasoning you used to answer the question. "
                    "Keep it concise and factual.\n\nQuestion: " + q,
                )["answer"]
                sections["REASONING"] = rationale
                reason_ms = int((time.time() - t_r) * 1000)

            progress(0.60, desc="Answering")
            t0 = time.time()

            # Try to ground the model with a quick caption, then ask the steered question.
            ans = ""
            try:
                cap = MODEL.caption(base)["caption"].strip()
                if cap:
                    primed_q = f"Image summary: {cap}\n\n{_steer(q)}"
                    ans = MODEL.query(base, primed_q)["answer"].strip()
            except Exception:
                pass

            # Fallback: ask the steered question directly.
            if not ans:
                ans = MODEL.query(base, _steer(q))["answer"].strip()

            # Retry once if the reply is low-information (e.g., "This").
            if _is_low_info(ans):
                ans = MODEL.query(
                    base,
                    _steer(q) + "\n\nBe specific and name the object/place. Avoid pronouns.",
                )["answer"].strip()

            meta = {
                "ms": int((time.time() - t0) * 1000) + reason_ms,
                "source": SOURCE,
            }
            sections["RESULT"] = ans.strip()

            payload = {
                "mode": "query",
                "prompt": q,
                "reasoning": sections.get("REASONING", ""),
                "result": ans,
                "meta": meta,
            }
            right = _panel_sections("query", sections, meta)
            return _updates(
                img_value=base,
                panel_html=right,
                code_html=make_code_details(jdump(payload)),
                show=True,
            )

        # --- CAPTION ---
        if mode == "Caption":
            progress(0.35, desc="Generating caption")
            caption_text, meta = caption_with_length(base, cap_len)
            sections = {"RESULT": caption_text}
            payload = {
                "mode": "caption",
                "text": caption_text,
                "meta": meta,
                "length": cap_len,
            }
            right = _panel_sections(
                "caption", sections, meta,
                extra_chip=f"LENGTH: {cap_len.upper()}",
            )
            return _updates(
                img_value=base,
                panel_html=right,
                code_html=make_code_details(jdump(payload)),
                show=True,
            )

        # --- POINT ---
        if mode == "Point":
            label = (labels or "object").strip()
            progress(0.4, desc=f"Locating '{label}'")
            t0 = time.time()
            pts_norm = MODEL.point(base, label)["points"]  # normalized
            meta = {
                "ms": int((time.time() - t0) * 1000),
                "source": SOURCE,
                "points": len(pts_norm),
            }
            pts_px = [{"xy": to_px_point(p, W, H)} for p in pts_norm]
            overlay = draw_overlay(base, detections=[], points=pts_px)
            summary = f"Found {len(pts_norm)} point(s) for '{label}'."
            sections = {"RESULT": summary}
            payload = {"mode": "point", "points": pts_norm, "meta": meta}
            right = _panel_sections("point", sections, meta)
            return _updates(
                img_value=overlay,
                panel_html=right,
                code_html=make_code_details(jdump(payload)),
                show=True,
            )

        # --- DETECT ---
        lbs = [t.strip() for t in (labels or "").split(",") if t.strip()] or ["object"]
        progress(0.4, desc=f"Detecting: {', '.join(lbs)}")
        t0 = time.time()
        all_norm = []
        for lb in lbs:
            res = MODEL.detect(base, lb)
            for o in res["objects"]:
                all_norm.append({"label": lb, **o})

        meta = {
            "ms": int((time.time() - t0) * 1000),
            "source": SOURCE,
            "detections": len(all_norm),
        }
        all_px = [{"label": d["label"], "box": to_px_box(d, W, H)} for d in all_norm]
        overlay = draw_overlay(base, detections=all_px, points=[])
        summary = f"Detections for: {', '.join(lbs)} ({len(all_norm)} total)."
        sections = {"RESULT": summary}
        payload = {"mode": "detect", "detections": all_norm, "meta": meta}
        right = _panel_sections("detect", sections, meta)
        return _updates(
            img_value=overlay,
            panel_html=right,
            code_html=make_code_details(jdump(payload)),
            show=True,
        )

    except Exception as e:
        payload = {"error": str(e)}
        right = (
            "<div class='panel'><div class='chips'>"
            "<span class='chip'>ERROR</span></div>"
            f"<div class='body'>Runtime error: {html.escape(str(e))}</div></div>"
        )
        return _updates(
            img_value=img,
            panel_html=right,
            code_html=make_code_details(jdump(payload)),
            show=True,
        )


In [ ]:
custom_css = r"""
/* =========================================================
   GLOBAL BACKGROUND / RESET
   ========================================================= */

html, body, .gradio-container {
  height: 100%;
  margin: 0;
}

.gradio-container {
  min-height: 100vh;
  max-width: none !important;
  margin: 0 !important;
  padding: 0 !important;
  box-sizing: border-box;
  background:
    radial-gradient(circle at top left, rgba(239, 68, 68, 0.3), transparent 55%),
    radial-gradient(circle at bottom right, rgba(251, 146, 60, 0.28), transparent 55%),
    linear-gradient(135deg, #fef2f2, #fee2e2, #fecaca);
  font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI",
               Roboto, "Helvetica Neue", Arial, sans-serif;
  position: relative;
  overflow: hidden;
}

.gradio-container::before {
  content: "";
  position: absolute;
  top: -50%;
  left: -50%;
  width: 200%;
  height: 200%;
  background: 
    radial-gradient(circle at 20% 50%, rgba(220, 38, 38, 0.15) 0%, transparent 50%),
    radial-gradient(circle at 80% 80%, rgba(239, 68, 68, 0.12) 0%, transparent 50%),
    radial-gradient(circle at 40% 20%, rgba(251, 113, 133, 0.1) 0%, transparent 50%);
  animation: gradientShift 15s ease infinite;
  pointer-events: none;
  z-index: 0;
}

.gradio-container > * {
  position: relative;
  z-index: 1;
}

@keyframes gradientShift {
  0%, 100% {
    transform: translate(0, 0) rotate(0deg);
    opacity: 1;
  }
  33% {
    transform: translate(-5%, 5%) rotate(1deg);
    opacity: 0.9;
  }
  66% {
    transform: translate(5%, -5%) rotate(-1deg);
    opacity: 0.95;
  }
}

/* =========================================================
   OUTER SHELL & MAIN CARD (CENTERED RECTANGLE)
   ========================================================= */

.app-shell {
  width: 100%;
  max-width: 1440px;
  margin: 0 auto;
  padding: 40px 32px;
  box-sizing: border-box;
  min-height: calc(100vh - 80px);
  display: flex !important;
  align-items: center;
  justify-content: center;
}

@media (max-width: 900px) {
  .app-shell {
    align-items: flex-start;
    padding: 24px 16px;
  }
}

.app-shell > .gr-column {
  flex: 0 0 auto !important;
  height: auto !important;
  min-height: 0 !important;
}

/* White rectangle card */
.page-card {
  background: #ffffff;
  border-radius: 24px;
  box-shadow: 0 22px 60px rgba(15,23,42,.20);
  padding: 48px 64px 56px;
  max-width: 1600px;
  width: 95%;
  margin: 0 auto;
  height: auto !important;
  min-height: 0 !important;
  max-height: fit-content !important;
}

/* Stop Gradio from forcing full-height on inner elements */
.page-card > .gr-column,
.page-card > .gr-group,
.page-card > .gr-row,
.page-card .gr-block,
.page-card .gr-box,
.page-card .gr-panel {
  height: auto !important;
  min-height: 0 !important;
  flex: 0 0 auto !important;
  flex-grow: 0 !important;
}

/* =========================================================
   HERO HEADER
   ========================================================= */

.hero-chip {
  display: inline-flex;
  align-items: center;
  justify-content: center;
  padding: 4px 14px;
  border-radius: 999px;
  background: #eef2ff;
  color: #6366f1;
  font-size: 12px;
  font-weight: 600;
  letter-spacing: .08em;
  text-transform: uppercase;
  margin: 0 auto 18px;
  width: fit-content;
}

.hero-title {
  font-size: 48px;
  line-height: 1.05;
  font-weight: 800;
  color: #0f172a;
  margin: 0 0 14px;
  text-align: center;
}

.hero-subtitle {
  margin: 0 0 36px;
  font-size: 16px;
  line-height: 1.7;
  color: #4b5563;
  text-align: center;
  max-width: 900px;
  margin-left: auto;
  margin-right: auto;
}

/* =========================================================
   3-STEP ROW
   ========================================================= */

.hero-steps {
  display: grid;
  grid-template-columns: repeat(3, minmax(0, 1fr));
  gap: 24px;
  margin-bottom: 32px;
}

.hero-step {
  display: flex;
  gap: 12px;
  align-items: flex-start;
  background: #f8fafc;
  border-radius: 16px;
  border: 1px solid #e2e8f0;
  padding: 18px 20px;
}

@media (max-width: 900px) {
  .hero-steps {
    grid-template-columns: 1fr;
  }
}

.hero-step-num {
  flex-shrink: 0;
  width: 26px;
  height: 26px;
  border-radius: 999px;
  background: #2563eb;
  color: #f9fafb;
  display: flex;
  align-items: center;
  justify-content: center;
  font-size: 13px;
  font-weight: 600;
  box-shadow: 0 8px 18px rgba(37,99,235,.35);
}

.hero-step-body h3 {
  margin: 0 0 4px;
  font-size: 14px;
  font-weight: 700;
  color: #0f172a;
}

.hero-step-body p {
  margin: 0;
  font-size: 13px;
  color: #6b7280;
  line-height: 1.45;
}

/* =========================================================
   HOME BUTTON ROW
   ========================================================= */

.hero-buttons {
  display: flex !important;
  justify-content: center;
  align-items: center;
  flex-wrap: wrap;
  gap: 14px;
  margin-top: 10px;
}

.hero-buttons > * {
  flex: 0 0 auto !important;
  width: auto !important;
}

.hero-btn {
  border-radius: 999px !important;
  padding: 11px 22px !important;
  font-size: 14px !important;
  font-weight: 600 !important;
  border: 0 !important;
  min-width: 0 !important;
  cursor: pointer !important;
  box-shadow: 0 12px 26px rgba(15,23,42,.18) !important;
}

.hero-btn--primary {
  background: #2563eb !important;
  color: #f9fafb !important;
}

.hero-btn--primary:hover {
  filter: brightness(1.02);
}

.hero-btn--ghost {
  background: #ecfdf5 !important;
  color: #0f172a !important;
  box-shadow: 0 8px 20px rgba(16,185,129,.16) !important;
}

.hero-btn--ghost:hover {
  filter: brightness(1.01);
}

/* =========================================================
   PLAYGROUND CARD WRAPPER
   ========================================================= */

.play-card {
  background: #ffffff;
  border-radius: 14px;
  box-shadow: 0 10px 24px rgba(2,6,23,.06);
  padding: 16px;
}

.page-card .play-card > .gr-row,
.page-card .play-card > .gr-column {
  width: 100%;
}

/* =========================================================
   PROMPT SURFACE
   ========================================================= */

:root {
  --arrow-size: 40px;
  --arrow-gap: 18px;
}

.prompt-surface {
  display: flex !important;
  flex-direction: column;
  border: 1px solid #e5e7eb;
  border-radius: 12px;
  background: #ffffff;
  box-shadow: 0 1px 2px rgba(0,0,0,.04);
  position: relative;
}

.prompt-input,
.prompt-input > div,
.prompt-input .wrap {
  margin: 0 !important;
}

.prompt-input textarea {
  border: 0 !important;
  box-shadow: none !important;
  background: #ffffff !important;
  padding: 14px 16px 30px 16px !important;
  resize: vertical;
  min-height: 160px;
  border-top-left-radius: 12px;
  border-top-right-radius: 12px;
}

/* =========================================================
   PROMPT FOOTER
   ========================================================= */

.prompt-footer {
  display: flex !important;
  align-items: center;
  flex-wrap: nowrap;
  gap: 16px;
  padding: 6px 12px;
  border-top: 0 !important;
  border-bottom-left-radius: 12px;
  border-bottom-right-radius: 12px;
  background: transparent !important;
  position: relative;
  padding-right: calc(var(--arrow-size) + var(--arrow-gap) + 6px);
  overflow: hidden;
  margin-top: 4px;
}

.prompt-footer > div {
  background: transparent !important;
  border: 0 !important;
  box-shadow: none !important;
}

.prompt-footer .controls {
  display: inline-flex !important;
  align-items: center;
  gap: 12px;
  flex: 0 1 auto;
  white-space: nowrap;
  min-width: 0;
}

/* Mode radio as pills */
.prompt-footer .mode-strip {
  display: flex !important;
  align-items: center;
  gap: 8px;
  white-space: nowrap;
  min-width: 0;
  flex: 0 0 auto;
}

.mode-strip input[type="radio"] {
  position: absolute;
  opacity: 0;
  pointer-events: none;
  width: 0;
  height: 0;
}

.mode-strip label {
  background: transparent;
  border: 1px solid transparent;
  border-radius: 10px;
  padding: 4px 10px;
  cursor: pointer;
  user-select: none;
  font-weight: 500;
  white-space: nowrap;
}

.mode-strip input[type="radio"]:checked + label {
  background: #ffffff;
  border-color: #d1d5db;
}

.mode-strip label:hover {
  filter: brightness(0.98);
}

.mode-strip label:focus-visible {
  outline: 2px solid #94a3b8;
  outline-offset: 2px;
}

/* Reasoning toggle */
.switch {
  display: inline-flex !important;
  align-items: center;
  gap: 8px;
  white-space: nowrap;
  flex: 0 0 auto;
  margin: 0;
}

.switch label {
  display: flex;
  align-items: center;
  gap: 8px;
  cursor: pointer;
  font-weight: 500;
  margin: 0;
}

.switch input[type="checkbox"] {
  appearance: none;
  width: 36px;
  height: 20px;
  border-radius: 999px;
  background: #e5e7eb;
  border: 1px solid #d1d5db;
  position: relative;
  outline: none;
  cursor: pointer;
}

.switch input[type="checkbox"]::after {
  content: "";
  position: absolute;
  top: 1px;
  left: 1px;
  width: 16px;
  height: 16px;
  border-radius: 50%;
  background: #ffffff;
  box-shadow: 0 1px 2px rgba(0,0,0,.2);
  transition: left .15s ease;
}

.switch input[type="checkbox"]:checked {
  background: #111827;
}

.switch input[type="checkbox"]:checked::after {
  left: calc(100% - 17px);
}

/* Arrow submit button */
.prompt-footer .gr-button,
.prompt-footer button {
  min-width: 0 !important;
  padding: 0 !important;
  box-shadow: none !important;
  line-height: 1 !important;
}

.submit-arrow {
  width: var(--arrow-size) !important;
  height: var(--arrow-size) !important;
  border-radius: 9999px !important;
  border: 1px solid #d1d5db !important;
  background: #ffffff !important;
  display: inline-flex !important;
  align-items: center !important;
  justify-content: center !important;
  font-size: 18px !important;
  line-height: 1 !important;
  position: absolute !important;
  right: var(--arrow-gap) !important;
  top: 50% !important;
  transform: translateY(-50%) !important;
  cursor: pointer;
}

.submit-arrow:hover {
  filter: brightness(0.97);
}

.submit-arrow.loading {
  pointer-events: none;
  color: transparent;
}

.submit-arrow.loading::after {
  content: "";
  position: absolute;
  left: 50%;
  top: 50%;
  transform: translate(-50%, -50%);
  width: 12px;
  height: 12px;
  border-radius: 50%;
  border: 2px solid #111827;
  border-right-color: transparent;
  animation: spin .9s linear infinite;
}

@keyframes spin {
  to { transform: translate(-50%, -50%) rotate(360deg); }
}

/* =========================================================
   OUTPUT PANEL
   ========================================================= */

.panel .chips {
  margin-bottom: 8px;
}

.chip {
  display: inline-block;
  font-weight: 600;
  font-size: 12px;
  letter-spacing: .02em;
  padding: 3px 8px;
  border-radius: 4px;
  background: #f3f4f6;
  color: #111827;
  border: 1px solid #d1d5db;
  margin-right: 6px;
}

.chip--muted {
  color: #334155;
  border: 1px solid #94a3b8;
  background: transparent;
}

.panel .section-h {
  font-weight: 700;
  text-transform: uppercase;
  font-size: 12px;
  color: #111827;
  margin: 8px 0 4px;
}

.gradio-container .panel .body,
.gradio-container .panel .body * {
  white-space: pre-line;
  line-height: 1.6;
  color: #111827;
  font-size: 15px !important;
  font-weight: 400 !important;
  font-family: inherit !important;
}

.panel .meta {
  color: #6b7280;
  font-size: 12px;
  margin-top: 12px;
}

/* =========================================================
   SUGGESTION CHIPS
   ========================================================= */

.sugg-row {
  position: relative;
  display: flex !important;
  justify-content: center;
  align-items: center;
  gap: 8px;
  margin: -24px auto 6px;
  width: fit-content;
  background: transparent !important;
  border: 0 !important;
  box-shadow: none !important;
  z-index: 2;
  flex-wrap: wrap;
}

.suggest-pill {
  display: inline-flex !important;
  align-items: center;
  justify-content: center;
  padding: 4px 12px !important;
  border-radius: 9999px !important;
  background: #f5f5f5 !important;
  border: 1px solid #e5e7eb !important;
  box-shadow: none !important;
  color: #4b5563 !important;
  font-size: 13px !important;
  font-weight: 400 !important;
  cursor: pointer !important;
  min-width: 0 !important;
  max-width: 260px;
  white-space: normal !important;
  text-align: center;
}

.suggest-pill:hover {
  background: #e5e7eb !important;
}

.suggest-pill:focus-visible {
  outline: 2px solid #94a3b8 !important;
  outline-offset: 2px !important;
}

.suggest-pill + .suggest-pill {
  margin-left: 8px;
}

.suggest-pill.loading {
  opacity: 0.65 !important;
  pointer-events: none !important;
  font-style: italic !important;
}

/* =========================================================
   IMAGE UPLOAD – MAKE IMAGE FILL ITS FRAME
   ========================================================= */

div[data-testid="image"] {
  width: 100% !important;
  height: 100% !important;
  display: flex !important;
  align-items: center !important;
  justify-content: center !important;
  background: #f8fafc;
  border-radius: 12px;
  overflow: hidden;
}

div[data-testid="image"] img,
div[data-testid="image"] canvas {
  width: 100% !important;
  height: auto !important;
  max-height: 100% !important;
  object-fit: contain !important;
  display: block !important;
}

/* =========================================================
   SAMPLE IMAGE STRIP – version you liked
   ========================================================= */

.sample-gallery {
  margin: 4px 0 12px 0;
  padding: 0 !important;
  border: 0 !important;
  box-shadow: none !important;
  background: transparent !important;
  height: auto !important;
  max-height: none !important;
  overflow: visible !important;
}

.sample-gallery > div,
.sample-gallery > div > div {
  padding: 0 !important;
  margin: 0 !important;
  border: 0 !important;
  box-shadow: none !important;
  background: transparent !important;
  height: auto !important;
  max-height: none !important;
  overflow: visible !important;
}

.sample-gallery div[style*="overflow"] {
  height: auto !important;
  max-height: none !important;
  overflow: visible !important;
}

.sample-gallery [data-testid="gallery"] {
  display: flex !important;
  flex-wrap: nowrap !important;
  gap: 10px;
  padding: 0 !important;
  margin: 0 !important;
  height: auto !important;
  max-height: none !important;
  border: 0 !important;
  box-shadow: none !important;
  background: transparent !important;
}

.sample-gallery [data-testid="gallery-item"] {
  flex: 0 0 120px !important;
  height: 80px !important;
  display: flex !important;
  align-items: center !important;
  justify-content: center !important;
  border-radius: 12px !important;
  overflow: hidden !important;
  border: 1px solid #e5e7eb !important;
  background: #f9fafb !important;
  padding: 0 !important;
}

.sample-gallery img {
  width: 100% !important;
  height: 100% !important;
  object-fit: cover !important;
  display: block !important;
  cursor: pointer !important;
}
"""


In [ ]:
with gr.Blocks(css=custom_css) as demo:
    # Outer gradient shell
    with gr.Column(elem_classes=["app-shell"]):

        # =========================================================
        # HOME (hero)
        # =========================================================
        with gr.Column(visible=True,
                       elem_classes=["page-card"]) as home_page:

            gr.HTML(
                """
<div style="text-align: center;">
  <div class="hero-chip">NASA AI LAB</div>
</div>
<h1 class="hero-title">ImageTagger Studio</h1>
<p class="hero-subtitle">
Explore NASA imagery with vision models and ChatGPT.
Choose a workspace to start tagging, searching, or reasoning over your images.
</p>

<div class="hero-steps">
  <div class="hero-step">
    <div class="hero-step-num">1</div>
    <div class="hero-step-body">
      <h3>Pick a workspace</h3>
      <p>Start in the vision reasoning agent or jump straight into the Moondream playground.</p>
    </div>
  </div>

  <div class="hero-step">
    <div class="hero-step-num">2</div>
    <div class="hero-step-body">
      <h3>Load your data</h3>
      <p>Upload images or connect to NASA data sources in the future.</p>
    </div>
  </div>

  <div class="hero-step">
    <div class="hero-step-num">3</div>
    <div class="hero-step-body">
      <h3>Explore & iterate</h3>
      <p>Ask questions, tag content, and refine prompts for your workflows.</p>
    </div>
  </div>
</div>
                """
            )

            with gr.Row(elem_classes=["hero-buttons"]):
                home_to_agent = gr.Button(
                    "🤖 Vision Reasoning Agent",
                    scale=0,
                    elem_classes=["hero-btn", "hero-btn--primary"],
                )
                home_to_moon = gr.Button(
                    "🖼️ Open Moondream Playground",
                    scale=0,
                    elem_classes=["hero-btn", "hero-btn--ghost"],
                )

        # =========================================================
        # VISION REASONING AGENT PAGE
        # =========================================================
        with gr.Column(visible=False,
                       elem_classes=["page-card"]) as vision_agent_page:

            with gr.Row():
                back_from_agent = gr.Button("← Back to home", scale=0)

            gr.Markdown("## Vision Reasoning Agent")

            gr.Markdown("**Sample Images**")
            agent_sample_gallery = gr.Gallery(
                value=SAMPLE_IMAGES,
                show_label=False,
                columns=len(SAMPLE_IMAGES),
                rows=1,
                allow_preview=False,
                elem_classes=["sample-gallery"],
            )

            with gr.Row(elem_classes=["play-card"]):
                with gr.Column(scale=1):
                    agent_img = gr.Image(
                        type="pil",
                        sources=["upload"],
                        image_mode="RGB",
                        label="Image",
                        height=320,
                        show_label=True,
                        interactive=True,
                    )

                with gr.Column(scale=2):
                    vision_question = gr.Textbox(
                        label="Ask a higher-level question about this scene",
                        placeholder=(
                            "Examples:\n"
                            "• Based on this picture, give your best estimate of how accessible this area is by car."
                            "• Does this scene look safe for a small field trip of kids?"
                            "• What natural hazards might be present here, and how confident are you?"
                        ),
                        lines=6,
                    )
                    run_agent_btn = gr.Button("Run vision reasoning")
                    vision_answer = gr.Markdown(
                        value="",
                        label="Reasoning & answer",
                    )

        # =========================================================
        # MOONDREAM PLAYGROUND PAGE (existing UI)
        # =========================================================
        with gr.Column(visible=False,
                       elem_classes=["page-card"]) as moondream_page:

            with gr.Row():
                back_from_moon = gr.Button("← Back to home", scale=0)

            gr.Markdown("## Moondream Playground")

            gr.Markdown("**Sample Images**")
            moon_sample_gallery = gr.Gallery(
                value=SAMPLE_IMAGES,
                show_label=False,
                columns=len(SAMPLE_IMAGES),
                rows=1,
                allow_preview=False,
                elem_classes=["sample-gallery"],
            )

            # ---------- TOP: two columns ----------
            with gr.Row(elem_classes=["play-card"]):
                # Left: image
                with gr.Column(scale=1):
                    img_in = gr.Image(
                        type="pil",
                        sources=["upload"],
                        image_mode="RGB",
                        label=None,
                        show_label=False,
                        interactive=True,
                        height = 320,
                    )

                # Right: prompt card
                with gr.Column(scale=2):
                    with gr.Group(elem_classes=["prompt-surface"]):
                        # text area (shown for Query/Point/Detect)
                        with gr.Group(visible=True) as prompt_wrap:
                            question = gr.Textbox(
                                label=None,
                                placeholder="Enter a question…",
                                lines=7,
                                show_label=False,
                                container=False,
                                elem_classes=["prompt-input"],
                            )

                        # suggestions row (starts hidden)
                        with gr.Row(
                            visible=False, elem_classes=["sugg-row"]
                        ) as sugg_row:
                            sugg_btn1 = gr.Button(
                                "", visible=False, elem_classes=["suggest-pill"]
                            )
                            sugg_btn2 = gr.Button(
                                "", visible=False, elem_classes=["suggest-pill"]
                            )
                            sugg_btn3 = gr.Button(
                                "", visible=False, elem_classes=["suggest-pill"]
                            )

                        # footer ...
                        with gr.Row(elem_classes=["prompt-footer"]):
                            with gr.Row(elem_classes=["controls"]):
                                mode_radio = gr.Radio(
                                    ["Query", "Caption", "Point", "Detect"],
                                    value="Query",
                                    show_label=False,
                                    container=False,
                                    interactive=True,
                                    elem_classes=["mode-strip"],
                                    scale=0,
                                )

                                reasoning = gr.Checkbox(
                                    label="Reasoning",
                                    value=False,
                                    container=False,
                                    show_label=False,
                                    elem_classes=["switch"],
                                    visible=True,   # Query is default
                                    scale=0,
                                )

                            submit_btn = gr.Button(
                                "➜",
                                elem_classes=["submit-arrow"],
                                scale=0,
                            )

                    # caption length row
                    with gr.Row(visible=False) as opts_caption:
                        cap_len = gr.Radio(
                            [("Short", "short"),
                             ("Normal", "normal"),
                             ("Long", "long")],
                            value="normal",
                            label="Caption length",
                        )

            # ---------- BOTTOM: results ----------
            gr.Markdown("## Results", elem_classes=["results-heading"])
            with gr.Row(elem_classes=["play-card"]):
                with gr.Column(scale=1):
                    out_img = gr.Image(label=None, height=320, visible=False)
                    code_html = gr.HTML(visible=False)
                with gr.Column(scale=1):
                    out_panel = gr.HTML(visible=False)

    # =========================================================
    # LOGIC / CALLBACKS (outside columns but inside Blocks)
    # =========================================================
    def _run_vision_agent(img, q):
        return run_vision_agent(img, q)

    run_agent_btn.click(
        _run_vision_agent,
        inputs=[agent_img, vision_question],
        outputs=[vision_answer],
    )

    def _on_mode_change(mode):
        show_prompt = mode in ("Query", "Point", "Detect")
        placeholder = {
            "Query": "Enter a question…",
            "Point": "Type the object to point at (e.g., 'grapes')",
            "Detect": "Comma-separated labels (e.g., 'bottle, grapes')",
            "Caption": "",
        }[mode]
        return (
            gr.update(visible=show_prompt),          # prompt_wrap
            gr.update(
                placeholder=placeholder,
                value="" if mode == "Caption" else None,
            ),                                       # question
            gr.update(visible=(mode == "Caption")),  # opts_caption
            gr.update(visible=(mode == "Query")),    # reasoning
        )

    mode_radio.change(
        _on_mode_change,
        mode_radio,
        [prompt_wrap, question, opts_caption, reasoning],
    )

    def _hide_results_on_new_image(_):
        return (
            gr.update(value=None, visible=False),  # out_img
            gr.update(value="", visible=False),    # out_panel
            gr.update(value="", visible=False),    # code_html
        )

    img_in.change(
        _hide_results_on_new_image,
        img_in,
        [out_img, out_panel, code_html],
    )

    def _start_sugg_loading(img, mode):
        row_hidden = gr.update(visible=False)
        empty_btn  = gr.update(value="", visible=False, interactive=False)

        if img is None or mode != "Query":
            return row_hidden, empty_btn, empty_btn, empty_btn

        row_shown    = gr.update(visible=True)
        loading_pill = gr.update(
            value="Generating suggestions…",
            visible=True,
            interactive=False,
            elem_classes=["suggest-pill", "loading"],
        )
        return (row_shown, loading_pill, empty_btn, empty_btn)

    def _update_suggestions(img, mode):
        row_hidden = gr.update(visible=False)
        empty_btn  = gr.update(value="", visible=False)

        if img is None or mode != "Query":
            return row_hidden, empty_btn, empty_btn, empty_btn

        fallback_qs = [
            "What are the main objects in this image?",
            "What is the setting or environment?",
            "Is there anything unusual or interesting here?",
        ]

        try:
            base = exif_rgb(img)
            qs = suggest_questions(base, n=3)
        except Exception:
            qs = fallback_qs

        if not qs:
            qs = fallback_qs

        updates = []
        for q in qs + [""] * (3 - len(qs)):
            if q:
                updates.append(
                    gr.update(
                        value=q,
                        visible=True,
                        interactive=True,
                        elem_classes=["suggest-pill"],
                    )
                )
            else:
                updates.append(
                    gr.update(value="", visible=False, interactive=False)
                )

        row_shown = gr.update(visible=True)
        return (row_shown, *updates)

    img_in.change(
        _start_sugg_loading,
        [img_in, mode_radio],
        [sugg_row, sugg_btn1, sugg_btn2, sugg_btn3],
    ).then(
        _update_suggestions,
        [img_in, mode_radio],
        [sugg_row, sugg_btn1, sugg_btn2, sugg_btn3],
    )

    mode_radio.change(
        _start_sugg_loading,
        [img_in, mode_radio],
        [sugg_row, sugg_btn1, sugg_btn2, sugg_btn3],
    ).then(
        _update_suggestions,
        [img_in, mode_radio],
        [sugg_row, sugg_btn1, sugg_btn2, sugg_btn3],
    )

        # --- Sample image handlers ---

    def _load_sample_for_moondream(evt: gr.SelectData):
        idx = evt.index  # which thumbnail was clicked
        # return the corresponding PIL image
        return SAMPLE_IMAGES[idx]

    moon_sample_gallery.select(
        _load_sample_for_moondream,
        inputs=None,
        outputs=[img_in],
    )

    def _load_sample_for_agent(evt: gr.SelectData):
        idx = evt.index
        return SAMPLE_IMAGES[idx]

    agent_sample_gallery.select(
        _load_sample_for_agent,
        inputs=None,
        outputs=[agent_img],
    )

    def _use_suggestion(text):
        return gr.update(value=text)

    sugg_btn1.click(_use_suggestion, inputs=sugg_btn1, outputs=question)
    sugg_btn2.click(_use_suggestion, inputs=sugg_btn2, outputs=question)
    sugg_btn3.click(_use_suggestion, inputs=sugg_btn3, outputs=question)

    def _run_bridge(mode, img, text, reasoning_val, cap_len,
                    progress=gr.Progress(track_tqdm=True)):
        if mode != "Query":
            reasoning_val = False

        if mode == "Point":
            labels = text or ""
        elif mode == "Detect":
            labels = text or ""
        else:
            labels = ""

        return run(mode, img, text, reasoning_val, cap_len, labels, progress)

    def _start_loading():
        return gr.update(elem_classes=["submit-arrow", "loading"])

    def _stop_loading():
        return gr.update(elem_classes=["submit-arrow"])

    submit_btn.click(
        _start_loading,
        inputs=None,
        outputs=[submit_btn],
    ).then(
        _run_bridge,
        [mode_radio, img_in, question, reasoning, cap_len],
        [out_img, out_panel, code_html],
    ).then(
        _stop_loading,
        inputs=None,
        outputs=[submit_btn],
    )

    question.submit(
        _start_loading,
        inputs=None,
        outputs=[submit_btn],
    ).then(
        _run_bridge,
        [mode_radio, img_in, question, reasoning, cap_len],
        [out_img, out_panel, code_html],
    ).then(
        _stop_loading,
        inputs=None,
        outputs=[submit_btn],
    )

    # --- Navigation between pages ---
    def show_home():
        return (
            gr.update(visible=True),   # home_page
            gr.update(visible=False),  # vision_agent_page
            gr.update(visible=False),  # moondream_page
        )

    def show_agent():
        return (
            gr.update(visible=False),
            gr.update(visible=True),
            gr.update(visible=False),
        )

    def show_moondream():
        return (
            gr.update(visible=False),
            gr.update(visible=False),
            gr.update(visible=True),
        )


    home_to_agent.click(
        show_agent,
        None,
        [home_page, vision_agent_page, moondream_page],
    )
    home_to_moon.click(
        show_moondream,
        None,
        [home_page, vision_agent_page, moondream_page],
    )

    back_from_agent.click(
        show_home,
        None,
        [home_page, vision_agent_page, moondream_page],
    )
    back_from_moon.click(
        show_home,
        None,
        [home_page, vision_agent_page, moondream_page],
    )


In [ ]:
# %% launch
try:
    demo.close()
except Exception:
    pass

demo.queue().launch(
    inline=True,
    show_error=True,
    server_name="127.0.0.1",
    server_port=None,
)